In [13]:
import pandas as pd

In [34]:
def time_check(df) -> pd.Series:
  # Function to check if a timestamp covers a full day
    def is_full_day(start_time, end_time):
        return (end_time - start_time).total_seconds() == 86400

    # Function to check if the given timestamps cover all 7 days of the week
    def is_full_week(start_day, end_day):
        return (end_day - start_day).days == 6

    # Convert the 'startDay' and 'endDay' columns to datetime objects
    df[['startDay', 'endDay']] = df[['startDay', 'endDay']].apply(pd.to_datetime)

    # Filter out unique (id, id_2) pairs
    unique_pairs = df[['id', 'id_2']].drop_duplicates()

    # Create a boolean series with multi-index (id, id_2)
    result = pd.Series(index=unique_pairs.index, dtype=bool)

    # Iterate over each unique (id, id_2) pair
    for pair in unique_pairs.index:
        # Filter out rows that belong to the current pair
        filtered_rows = df[(df['id'] == pair[0]) & (df['id_2'] == pair[1])]

        # Check if the timestamps cover a full 24-hour period for each day and span all 7 days of the week
        full_days = [is_full_day(filtered_rows.loc[i, 'startTime'], filtered_rows.loc[i, 'endTime']) for i in filtered_rows.index]
        full_weeks = [is_full_week(filtered_rows.loc[i, 'startDay'], filtered_rows.loc[i, 'endDay']) for i in filtered_rows.index]

        # If there is any day where the timestamps do not cover a full 24-hour period or any day that is not part of a full week, set the corresponding result to True
        if not all(full_days) or not all(full_weeks):
            result.loc[pair] = True

    return result

In [35]:
df = pd.read_csv('dataset-2.csv')
print(time_check(df))

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1-01-01 00:00:00 present at position 0